In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_interval = 250

In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    chars = f.read()
vocab_size = len(set(chars))

In [4]:
string_to_int= {ch:i for i,ch in enumerate(chars)}
int_to_string= {i:ch for i,ch in enumerate(chars)}
encode=lambda s: [string_to_int[c] for c in s]
decode=lambda l: ''.join([int_to_string[i] for i in l])
data=torch.tensor(encode(chars), dtype=torch.long, device=device)